In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [16]:
data = pd.read_csv('Landslide Civildf_new.csv')
data.head()

,Unnamed: 0,Timestamp,Rainfall,Water level,Inc30mup,Inc 30m down,Inc 20m up,Inc 20m down,Inc 10m up,Inc 10m down,1st Tensiometer,2nd Tensiometer,3rd Tensiometer,4th Tensiometer,5th Tensiometer
0,0,2022-12-01 01:00:00,0.0,-177.753910,24.520339,31.058241,16.023206,26.345443,20.637553,25.830825,-48.832508,-49.926434,181.57994,-3.156536,-48.597010
1,1,2022-12-01 02:00:00,0.0,-104.417316,26.093114,37.702314,15.116429,36.349078,20.836093,29.274305,-48.868384,-49.929076,181.60057,-3.142777,-48.629946
2,2,2022-12-01 03:00:00,0.0,-158.065690,29.426647,27.552242,20.673672,31.637946,20.879557,26.278678,-48.895732,-49.930162,181.61358,-3.129418,-48.652374
3,3,2022-12-01 04:00:00,0.0,-189.458110,30.382570,34.624852,21.397603,27.999477,22.426564,25.463844,-48.914294,-49.932082,181.61197,-3.137719,-48.671676
4,4,2022-12-01 05:00:00,0.0,-197.408960,25.493729,29.837320,18.103459,31.207890,23.932405,28.861409,-48.914736,-49.932394,181.62403,-3.137719,-48.673116


In [17]:
print(data.columns)

Index(['Unnamed: 0', 'Timestamp', 'Rainfall', 'Water level', 'Inc30mup',
       'Inc 30m down ', 'Inc 20m up ', 'Inc 20m down', 'Inc 10m up',
       'Inc 10m down', '1st Tensiometer', '2nd Tensiometer',
       '3rd Tensiometer ', '4th Tensiometer', '5th Tensiometer'],
      dtype='object')


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

data = pd.read_csv('Landslide Civildf_new.csv')

features = data[['Rainfall', 'Water level', 'Inc30mup', 'Inc 30m down ', 'Inc 20m up ', 'Inc 20m down', 'Inc 10m up', 'Inc 10m down', '1st Tensiometer', '2nd Tensiometer', '3rd Tensiometer ', '4th Tensiometer', '5th Tensiometer']]

# Normalizing
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

#training and testing sets
X_train, X_test = train_test_split(scaled_features, test_size=0.25, random_state=42)

# Define parameters
n_input = 30  # Number of time steps to look back 
n_features = features.shape[1]  # Number of features

# Create generator for training set
train_generator = TimeseriesGenerator(X_train, X_train, length=n_input, batch_size=1)

# Define the model
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model

model.fit(train_generator, epochs=100)


Epoch 1/100
492/492 [==============================] - 8s 11ms/step - loss: 0.0847
Epoch 2/100
492/492 [==============================] - 6s 12ms/step - loss: 0.0838
Epoch 3/100
492/492 [==============================] - 6s 12ms/step - loss: 0.0837
Epoch 4/100
492/492 [==============================] - 6s 12ms/step - loss: 0.0838
Epoch 5/100
492/492 [==============================] - 6s 12ms/step - loss: 0.0833
Epoch 6/100
492/492 [==============================] - 6s 12ms/step - loss: 0.0831
Epoch 7/100
492/492 [==============================] - 6s 12ms/step - loss: 0.0834
Epoch 8/100
492/492 [==============================] - 6s 13ms/step - loss: 0.0834
Epoch 9/100
492/492 [==============================] - 6s 12ms/step - loss: 0.0831
Epoch 10/100
492/492 [==============================] - 5s 11ms/step - loss: 0.0830
Epoch 11/100
492/492 [==============================] - 5s 10ms/step - loss: 0.0832
Epoch 12/100
492/492 [==============================] - 5s 11ms/step - loss: 0.0831
E

In [19]:
test_generator = TimeseriesGenerator(X_test, X_test, length=n_input, batch_size=1)

# Evaluate the model on the test set
loss = model.evaluate(test_generator)

print(f'Test loss: {loss}')

144/144 [==============================] - 2s 6ms/step - loss: 0.0855
Test loss: 0.08553926646709442


In [20]:
# Assume 'Rainfall' is the feature you're predicting
rainfall_scaler = MinMaxScaler()
rainfall_scaler.fit(data[['Rainfall']])

# When you make predictions
predictions = model.predict(test_generator)

# Use the 'rainfall_scaler' to inverse transform the predictions
predictions = rainfall_scaler.inverse_transform(predictions)

print(f'Predictions: {predictions}')

144/144 [==============================] - 2s 7ms/step
Predictions: [[0.61446315]
 [0.7864432 ]
 [0.7797791 ]
 [0.5967002 ]
 [0.67701554]
 [0.85784054]
 [0.64114827]
 [0.6615109 ]
 [0.7276707 ]
 [0.8812496 ]
 [0.78600955]
 [0.7363827 ]
 [0.7996854 ]
 [0.60604256]
 [0.5305718 ]
 [0.7767755 ]
 [0.62091833]
 [0.7230848 ]
 [0.77126443]
 [0.6656483 ]
 [0.49645054]
 [0.82682157]
 [0.69400036]
 [0.7245353 ]
 [0.6017488 ]
 [0.7528197 ]
 [0.5978551 ]
 [0.74524397]
 [0.67228246]
 [0.50605714]
 [0.5734782 ]
 [0.79047346]
 [0.6478131 ]
 [0.64771426]
 [0.7002743 ]
 [0.69058263]
 [0.73401976]
 [0.6221405 ]
 [0.6333508 ]
 [0.6572732 ]
 [0.70859706]
 [0.7244166 ]
 [0.7969896 ]
 [0.48728433]
 [0.77183425]
 [0.75631547]
 [0.7053447 ]
 [0.792289  ]
 [0.6229819 ]
 [0.77977455]
 [0.7547999 ]
 [0.4981776 ]
 [0.49974462]
 [0.7549223 ]
 [0.72505385]
 [0.90257657]
 [0.78822243]
 [0.82295525]
 [0.7020608 ]
 [0.73784375]
 [0.67263126]
 [0.65669644]
 [0.6271586 ]
 [0.73462754]
 [0.60257035]
 [0.7126217 ]
 [0.7591

In [40]:
model.save('my_model.h5')  # Creates a HDF5 file 'my_model.h5'


c:\Program Files\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
